# Asbestos Audit:
### outputs:
* ASB audit
* Duplicate APNs in share point, Tetra forms, Smart Sheets
* ASB sample sent prior to ASB assessment
* ASB results prior to ASB sample sent date


## Downloads needed:
* 2 Smartsheets both Central and South or BOP and ANVIL
* 3- Tetrat forms: Chimney tips, Abatment form, and survey form
* ESL SQL tracker
* Asbestos tracker - Sharepoint


# Libraries

In [ ]:
import pandas as pd
import glob
import numpy as np
import os
import smartsheet
import datetime

# Variables

In [ ]:
csv = '.csv'
excel = '.xlsx'

# Choose your Destiny(central or Northern Audits

In [ ]:
while True:
    user = input('What project are we Auditing?: Central or Northern ').lower()
    try:

        #central Set up
        if user == 'central':
            print('Central Audit Started')
            # Counties:
            county= ['ALPINE', 'EL DORADO', 'LAKE', 'NEVADA', 'PLACER', 'TUOLUMNE', 'MENDOCINO', 'TULARE', 'KERN']

            # Division. * northern goes here becasue we want everything not in northern Division
            division = 'Northern Division'

            # smartsheets Shet IDs
            sheet_id1 = '1844440261257092' # central
            sheet_id2= '6292262490531716' # south central

            # Central's API
            API1 = smartsheet.Smartsheet('6jJFxnAfFsDErU0BSr1Ae4D6cHq0zenczVR12') # API central

            API1.errors_as_exceptions(True)

            # Central's API
            API2 = smartsheet.Smartsheet("H8mqAo60J2zGHSrJEwmlMoY88A9yjgCo6RnSI") # API south central
            API2.errors_as_exceptions(True)
            break

        #northern Set up
        elif user == 'northern':
            print('Northern Audit Started')
            # Counties:
            county = ['LASSEN', 'PLUMAS', 'SISKIYOU', 'TRINITY', 'SHASTA', 'TEHAMA']

            # Division. * northern goes here becasue we want everything not in northern Division
            division = 'Central Division'

            # smartsheets Shet IDs
            sheet_id1 = '3589680131270532' # BOP
            sheet_id2= '5367491607193476' # ANVIL

            # Central's API
            API1 = smartsheet.Smartsheet('6w7vusWNdvdm9TJmjImZfbh0SPQgi3HJaekNb') # API BOP

            API1.errors_as_exceptions(True)

            # Central's API
            API2 = smartsheet.Smartsheet("MQQmCjvvFWLKBoX9K7A33rHSoLnuj1MGVJuVK") # API ANVIL
            API2.errors_as_exceptions(True)
            break
    except ValueError:
        print("\ninvaild input Only 'Central or Northern' allowed")

# Function to set up Smartsheet into a readable Database

In [ ]:
def smartsheet_loadup(sheet):
    columns = [col.title for col in sheet.columns]
    rows = []
    for row in sheet.rows:
        cells = []
        for cell in row.cells:
            cells.append(cell.value)
        rows.append(cells)
    df = pd.DataFrame(rows, columns=columns)
    return df

Area1_SS = smartsheet_loadup(API1.Sheets.get_sheet(sheet_id1))
Area2_SS = smartsheet_loadup(API2.Sheets.get_sheet(sheet_id2))

df_both = pd.concat([Area1_SS,Area2_SS])

# Read in Asbestos:
* Sharepoint tracker
* SQL ESL

In [ ]:
# Read in asbestos tracker from share point
path2 = r"C:\Users\jacque.trahan\Downloads\Asbestos Tracker*"
load_it2 = glob.glob(path2 + csv)

df2 = pd.read_csv(max(load_it2, key=os.path.getctime))

In [ ]:
# read in ESL from SQL server
path3 = r"C:\Users\jacque.trahan\Downloads\Asbestos Sample Status*"
load_it3 = glob.glob(path3 + excel)
esl_final = pd.read_excel(max(load_it3, key=os.path.getctime), header=1)

# Read in Tetra Froms

In [ ]:
# we have to open the file then save it with a different name as an excel. Find a way to solve this
df3= pd.read_excel("../../tetra forms ASB and SA/ASB_TetraForms_survey1.xlsx")

# Clean Data Bases 

In [ ]:
smartsheets = df_both.copy()
asbestos = df2.copy()
tetratForm = df3.copy()

In [ ]:
tetratForm = tetratForm[tetratForm['record_void'] != "Yes"]

# Most important info

What I need to Check Daily is Smart Sheets vs ENV. Trackers
* Chimneys and neshaps match
* FINAL RESULTS vs Smart sheets – biggest concern

* ASB Assessment Date Match

* End of day numbers match

# Asbestos set up

In [ ]:
# Take out all divisions other than central.
asbestos = asbestos[(~asbestos['Division'].isin([division]))]

###  Get APN counts from Asbetsos

In [ ]:
# pd.set_option('display.max_rows',1000)

In [ ]:
asbestos['APN'].value_counts().to_excel('Asbestos tracker APN Counts_Share Point-central.xlsx')

In [ ]:
# changes date column from object to a date.
asbestos['Date Collected'] = pd.to_datetime(asbestos['Date Collected']).dt.date.fillna(0)
asbestos['Chimney Tip Date'] = pd.to_datetime(asbestos['Chimney Tip Date']).dt.date.fillna(0)
asbestos['Chimney Abatement Completed Date'] = pd.to_datetime(asbestos['Chimney Abatement Completed Date']).dt.date.fillna(0)
asbestos['Asbestos Abatement Completed Date'] = pd.to_datetime(asbestos['Asbestos Abatement Completed Date']).dt.date.fillna(0)
asbestos['Date Shipped or Driven'] = pd.to_datetime(asbestos['Date Shipped or Driven']).dt.date.fillna("0")

In [ ]:
asbestos.columns = map(str.upper, asbestos.columns)

In [ ]:
asbestos.rename(columns={'APN':'APN_ASB'}, inplace=True)

In [ ]:
# Column section all upper case
asb_cols = asbestos[['APN_ASB', 'FINAL RESULTS', 'CHIMNEY', 'CHIMNEY TIP DATE', 'CHIMNEY FINDING',
                 'NESHAP WALLS',
                 'DATE COLLECTED', 'DATE SHIPPED OR DRIVEN', 
                 'SAMPLE QUANTITY', 'POINT COUNT NEEDED', 'POINT COUNT RESULTS',
                 'ASBESTOS ABATEMENT COMPLETED DATE', 'CHIMNEY ABATEMENT COMPLETED DATE',
                 'PLANNING NOTES', 'DIVISION']]

# Tetra Forms setup

## Tetra Forms chimney tip setup
Combine chimney tip database with sample log

In [ ]:
# Read in the chuimney tip database
TF_tips = pd.read_excel('../../tetra forms ASB and SA/ASB_TetraForms_tips1.xlsx')

In [ ]:
TF_tips = TF_tips[TF_tips['record_void'] != 'Yes']

In [ ]:
TF_tips = TF_tips[TF_tips['county'].isin(county)]

In [ ]:
# columns needed
TF_tips= TF_tips[['apn', 'inspection_date']]

### Get Chimney Tip APN counts in asbestos

In [ ]:
TF_tips['apn'].value_counts().to_excel('Asbestos Chimney tip APN counts.xlsx') # 1 for 1 last check all good

In [ ]:
# rename for easy reading for later merges
TF_tips.rename(columns={'inspection_date': 'TIP INSPECTION_TF'}, inplace=True)

In [ ]:
# Need to make this series a datetime and fill in 0'sfor blank spots
TF_tips['TIP INSPECTION_TF'] = pd.to_datetime(TF_tips['TIP INSPECTION_TF']).dt.date.fillna(0)

# Tetra Forms Abatment date setup
Combine Abatment date database with merge 1 (tips and max samples)

In [ ]:
# abatment clearance form logs
TF_abate = pd.read_excel('../../tetra forms ASB and SA/ASB_TetraForms_abatment1.xlsx')

In [ ]:
TF_abate = TF_abate[TF_abate['record_void'] != 'Yes']

In [ ]:
# chossing couty for only central division. Only way to seperate the divisions in Tetra forms
TF_abate = TF_abate[TF_abate['county'].isin(county)]

In [ ]:
# columns needed
TF_abate = TF_abate[['apn', 'abatement_end']]

#### Get Tetra Forms Abatment APN counts in asbestos

In [ ]:
TF_abate['apn'].value_counts().to_excel('TF Abatmanet APN Counts central.xlsx')

In [ ]:
# Need to make this series a datetime and fill in 0'sfor blank spots
TF_abate['abatement_end'] = pd.to_datetime(TF_abate['abatement_end']).dt.date.fillna(0)

In [ ]:
# rename for easy reading on later merges
TF_abate.rename(columns={'abatement_end': 'ABATEMENT END_TF'}, inplace=True)

In [ ]:
TF_tips_abate_merge = TF_tips.merge(TF_abate,
                     left_on='apn',
                     right_on='apn',
                     how='outer')

# Merge all Three Tetra Form Databases
* Chimney tips
* abatement dates
* asbestos survey info

In [ ]:
# merges in tetra froms surveys with sample logs, tips, and abataemetn databses.
tetratForms = tetratForm.merge(TF_tips_abate_merge,
                 left_on='apn',
                 right_on='apn',
                 how='left')

In [ ]:
# chossing couty for only central division. Only way to seperate the divisions in Tetra forms
tetratForms = tetratForms[tetratForms['county'].isin(county)]

In [ ]:
# columns needed
tetratForms = tetratForms[['apn', 'chimneys','walls','inspection_date', 'county','sample_counter_pass','TIP INSPECTION_TF',
                           'ABATEMENT END_TF' ]]

###  TF Survey APN counts

In [ ]:
tetratForms['apn'].value_counts().to_excel('Asbestos Survey TF_APN counts central.xlsx')

# Clean Tetra Form Data

In [ ]:
# change date object to date time format
tetratForms['inspection_date'] = pd.to_datetime(tetratForms['inspection_date']).dt.date.fillna(0)

In [ ]:
# rename out columns for easier readin on merges
tetratForms.rename(columns={'inspection_date': 'survey inspection date'}, inplace=True)

In [ ]:
# make ALL columnS upper CASE
tetratForms.columns =map(str.upper, tetratForms.columns)

In [ ]:
tetratForms.rename(columns={'APN': 'APN_TF'}, inplace=True)

# Smart Sheet Cleaning

### Get APN counts from Smart Sheets

In [ ]:
smartsheets['APN_ROW Segment'].value_counts().to_excel('Central_SS Tracker APN counts.xlsx')

#### This is need becasue BUG: pd.to_datetime() throws if caching is on with Null-like arguments
* solution: pd.to_datetime(df.colx.fillna(pd.NaT), errors='coerce')

In [ ]:
# smartsheets["Chimney Tipped"] = pd.to_datetime(smartsheets["Chimney Tipped"].fillna(pd.NaT), errors='coerce')
smartsheets["ASB Assessment"] = pd.to_datetime(smartsheets["ASB Assessment"].fillna(pd.NaT), errors='coerce')
smartsheets["ASB Abatement"] = pd.to_datetime(smartsheets["ASB Abatement"].fillna(pd.NaT), errors='coerce')
smartsheets['ASB Results Received'] = pd.to_datetime(smartsheets['ASB Results Received'].fillna(pd.NaT), errors='coerce')

In [ ]:
# changes date column from object to a date. and fill na with 0's
smartsheets["ASB Assessment"] = smartsheets["ASB Assessment"].dt.date.fillna(0)
# smartsheets["Chimney Tipped"] = smartsheets["Chimney Tipped"].dt.date.fillna(0)
smartsheets["ASB Abatement"] = smartsheets["ASB Abatement"].dt.date.fillna(0)
smartsheets['ASB Results Received'] = smartsheets['ASB Results Received'].dt.date.fillna(0)

In [ ]:
smartsheets.rename(columns={'ASB Assessment': 'Date Collected', 'Chimney Tipped':'CHIMNEY TIP DATE',
                            'County': 'SS_COUNTY'}, inplace=True)

In [ ]:
smartsheets.columns = map(str.upper, smartsheets.columns)

## Setup up columns and Dvision for Smartsheets

In [ ]:
# Column section normal uppercase
ss_cols = smartsheets[['APN_ROW SEGMENT','STREET #','STREET NAME','STRUCTURAL STATUS','SS_COUNTY',
                       'ASB RESULTS', 'ASB RESULTS RECEIVED', 
                     'CHIMNEY','CHIMNEY TIP DATE',
                     'NESHAP WALLS', 
                     "DATE COLLECTED",'ASB SUSPECTED',
                     'ASB ABATEMENT', 'DIVISION']]
ss_cols = ss_cols[~ss_cols['DIVISION'].isin([division])]

# Merge Smartsheets with Asbestos Tracker

In [ ]:
smasrt_ASB = ss_cols.merge(asb_cols,
                           left_on='APN_ROW SEGMENT',
                           right_on='APN_ASB',
                           how='left',
                         suffixes=('_SS','_ASB'))

# smartsheet w/ ASB merge with Tetraform

In [ ]:
SS_ASB_TF = smasrt_ASB.merge(tetratForms,
           left_on="APN_ROW SEGMENT",
           right_on='APN_TF',
           how='left',
           suffixes=("_SS", "_TF"))

# ESL Set up

In [ ]:
esl_final.columns = map(str.upper, esl_final.columns)

In [ ]:
esl_final.rename(columns={'DATE SHIPPED':'DATE SHIPPED_esl'}, inplace=True)

In [ ]:
esl_final = esl_final[~esl_final['DIVISION'].isin([division])]

In [ ]:
exceedances = esl_final[['APN', 'BULK PLM EXCEEDANCES']]

In [ ]:
esl_final = esl_final[['APN', 'DATE SHIPPED_esl','BULK PLM EXCEEDANCES', 'DIVISION']]

In [ ]:
esl_final['DATE SHIPPED_esl'] = pd.to_datetime(esl_final['DATE SHIPPED_esl']).dt.date.fillna("0")

In [ ]:
esl = esl_final.groupby(['APN','DATE SHIPPED_esl']).max().reset_index()
esl = esl[['APN','DATE SHIPPED_esl']]

# Merge SS_ASB_TF Merge with ESL master

In [ ]:
pre_final = exceedances.merge(esl,
            left_on="APN",
            right_on='APN',
            how='outer')

In [ ]:
final = SS_ASB_TF.merge(pre_final,
                       left_on="APN_ROW SEGMENT",
                       right_on='APN',
                       how='outer')

# set up columns for QC order

In [ ]:
# change the ND/POS to spell out for easier matching in audits
final['FINAL RESULTS'] = final['FINAL RESULTS'].str.replace("ND", 'NEGATIVE').str.replace("POS","POSITIVE")

# Get hacky and force blanks to say 0
this is beacsue some APN are not in every tracker yet

## Smartsheets 


In [ ]:
final["DATE COLLECTED_SS"] = final["DATE COLLECTED_SS"].fillna(0)

In [ ]:
final['CHIMNEY TIP DATE_SS'] = pd.to_datetime(final['CHIMNEY TIP DATE_SS'].fillna(pd.NaT), errors='coerce').dt.date

In [ ]:
final['CHIMNEY TIP DATE_SS'] = final['CHIMNEY TIP DATE_SS'].fillna(0)
final['ASB RESULTS'] = final['ASB RESULTS'].fillna("0")

final['CHIMNEY_SS'] = final['CHIMNEY_SS'].str.replace("No", '0')
final['CHIMNEY_SS'] = final['CHIMNEY_SS'].fillna("0")

final["ASB ABATEMENT"] = final["ASB ABATEMENT"].fillna(0)

final["NESHAP WALLS_SS"] = final["NESHAP WALLS_SS"].str.replace("No", "0").fillna("0")
final['ASB SUSPECTED'] = final['ASB SUSPECTED'].fillna("0")


## ASB Tracker


In [ ]:
final['DATE COLLECTED_ASB'] = final['DATE COLLECTED_ASB'].fillna(0)
final['DATE SHIPPED OR DRIVEN'] = final['DATE SHIPPED OR DRIVEN'].fillna("0")

final['CHIMNEY TIP DATE_ASB'] = final['CHIMNEY TIP DATE_ASB'].fillna(0)

final['SAMPLE QUANTITY'] = final['SAMPLE QUANTITY'].fillna(0)

final['FINAL RESULTS'] = final['FINAL RESULTS'].fillna("0")
final['POINT COUNT RESULTS'] = final['POINT COUNT RESULTS'].fillna("0")

final['CHIMNEY_ASB'] = final['CHIMNEY_ASB'].str.replace("No", "0")
final['CHIMNEY_ASB'] = final['CHIMNEY_ASB'].fillna('0')

final["ASBESTOS ABATEMENT COMPLETED DATE"] = final["ASBESTOS ABATEMENT COMPLETED DATE"].fillna(0)

# filled No with 0 becasue other databases do not put No they put 0
final["NESHAP WALLS_ASB"] = final["NESHAP WALLS_ASB"].str.replace("No", "0").fillna("0")


## Tetra Forms


In [ ]:
final['TIP INSPECTION_TF'] = final['TIP INSPECTION_TF'].fillna(0)

final['SURVEY INSPECTION DATE'] = final['SURVEY INSPECTION DATE'].fillna(0)

final['ABATEMENT END_TF'] = final['ABATEMENT END_TF'].fillna(0)

final['SAMPLE_COUNTER_PASS'] = final['SAMPLE_COUNTER_PASS'].fillna(0)

final['CHIMNEYS'] = final['CHIMNEYS'].str.replace('No', "0").fillna("0")


final["ABATEMENT END_TF"] = final["ABATEMENT END_TF"].fillna(0)

final["WALLS"] = final["WALLS"].str.replace("No", "0").fillna("0")


# ESL fillna with 0's

In [ ]:
final['DATE SHIPPED_esl'] = final['DATE SHIPPED_esl'].fillna("0")
final['DATE SHIPPED OR DRIVEN'] = final['DATE SHIPPED OR DRIVEN'].fillna("0")

In [ ]:
final = final[["APN_ROW SEGMENT",'APN_ASB','APN_TF','STREET #','STREET NAME','STRUCTURAL STATUS','SS_COUNTY',
               "DATE COLLECTED_SS",'DATE COLLECTED_ASB','SURVEY INSPECTION DATE',
               'SAMPLE QUANTITY','SAMPLE_COUNTER_PASS',
               'ASB SUSPECTED', 'DATE SHIPPED OR DRIVEN','DATE SHIPPED_esl',
               'ASB RESULTS','FINAL RESULTS','POINT COUNT NEEDED', 'POINT COUNT RESULTS', 'ASB RESULTS RECEIVED', 'BULK PLM EXCEEDANCES',
               'CHIMNEY_SS','CHIMNEY_ASB','CHIMNEYS',
               'CHIMNEY TIP DATE_SS','CHIMNEY TIP DATE_ASB','TIP INSPECTION_TF',
               'CHIMNEY FINDING', 'CHIMNEY ABATEMENT COMPLETED DATE',
               'ASB ABATEMENT','ASBESTOS ABATEMENT COMPLETED DATE', 'ABATEMENT END_TF',
               'NESHAP WALLS_SS','NESHAP WALLS_ASB','WALLS',
               'DIVISION_SS']]

In [ ]:
final = final.copy()

# If ASB is suspected in SS shipped should not be blank in ASB tracker

In [ ]:
def shipping_dates(df, comp1, comp2, comp3):
    if ((df[comp1] != "0") & ((df[comp2] == "0") | (df[comp3] == "0"))):
        return 0
    else:
        return 1

In [ ]:
final.insert(final.columns.get_loc('DATE SHIPPED_esl')+1, 'Blank Ship Date',
            final.apply(shipping_dates,comp1='ASB SUSPECTED', comp2='DATE SHIPPED OR DRIVEN', comp3='DATE SHIPPED_esl',axis=1))

# Automate the Excel QC process Function for all below process

In [ ]:
def compare_cols(df, comp1, comp2, comp3=None):
    if comp3 == None:
        if df[comp1] == df[comp2]:
            return 1
        else:
            return 0
    elif ((df[comp1] == df[comp2]) & (df[comp2] == df[comp3])):
        return 1
    else:
        return 0

# Check date shipped in ASB and date shipped in ESL

In [ ]:
final.insert(final.columns.get_loc('Blank Ship Date')+1, 'Check if ship dates Match',
             final.apply(compare_cols, comp1='DATE SHIPPED OR DRIVEN', comp2='DATE SHIPPED_esl', axis=1))

# check Date Collected

In [ ]:
final.insert(final.columns.get_loc('SAMPLE QUANTITY')+1,'Date Collected - M',
             final.apply(compare_cols, comp1="DATE COLLECTED_SS", comp2='DATE COLLECTED_ASB', comp3='SURVEY INSPECTION DATE', axis=1))

# check Total Samples Collected Counts

In [ ]:
final.insert(final.columns.get_loc('SAMPLE_COUNTER_PASS')+1, "Total Samples Collected Counts-M", 
            final.apply(compare_cols, comp1='SAMPLE QUANTITY', comp2='SAMPLE_COUNTER_PASS',axis=1))

In [ ]:
# have to use in place here or it will turn everythig to 0 if we use df[cols] = df[col].fillna(0) Like above
final['Total Samples Collected Counts-M'].fillna(0, inplace=True)

# check Final Results

Not working correctly. leaves a blank if there is an NS in column. Solve it

In [ ]:

final.insert(final.columns.get_loc('BULK PLM EXCEEDANCES')+1, "Final Results-M", 
            final.apply(compare_cols, comp1="ASB RESULTS", comp2="FINAL RESULTS", axis=1))

In [ ]:
final["Final Results-M"].fillna(0,inplace=True)

# Check Chimney 

In [ ]:
final.insert(final.columns.get_loc('CHIMNEYS')+1, 'Has Chimney-M',
            final.apply(compare_cols, comp1="CHIMNEY_SS", comp2="CHIMNEY_ASB", comp3="CHIMNEYS", axis=1))

In [ ]:
final['Has Chimney-M'].fillna(0, inplace=True)

# check CHIMNEY TIP DATE

In [ ]:
final.insert(final.columns.get_loc('TIP INSPECTION_TF')+1, "Chimney Tip dates-M", 
            final.apply(compare_cols, comp1="CHIMNEY TIP DATE_SS", comp2="CHIMNEY TIP DATE_ASB", comp3="TIP INSPECTION_TF", axis=1))

# check Abatements Dates

In [ ]:
final.insert(final.columns.get_loc('ABATEMENT END_TF')+1, "ASB ABATEMENT Date-M",
            final.apply(compare_cols,
                        comp1="ASB ABATEMENT", comp2="ASBESTOS ABATEMENT COMPLETED DATE", comp3="ABATEMENT END_TF", axis=1))

# check NESHAPS

In [ ]:
final.insert(final.columns.get_loc('WALLS')+1, "Neshap Wall-M",
            final.apply(compare_cols, comp1="NESHAP WALLS_SS", comp2="NESHAP WALLS_ASB", comp3="WALLS", axis=1))

# Highight columns

In [ ]:
# # set match columns to a certain color:
# # SYNTAX - df.style.set_properties(**{'background-color': 'color wanted'}, subset=["Col A", 'Col b', 'etc.'])

df_style = final.style.set_properties(**{'background-color': '#ffffb3'},
                           subset=['Date Collected - M',"Neshap Wall-M","ASB ABATEMENT Date-M",
                                  "Chimney Tip dates-M",'Has Chimney-M',
                                   "Final Results-M","Total Samples Collected Counts-M",
                                   'Blank Ship Date', 'Check if ship dates Match'], inplace=True)

# Make Smart sheets APNs the index and put in excel

In [ ]:
df_style.to_excel("Central Dixie ASB Audit.xlsx")

# ASB sample sent prior to ASB assessment

In [ ]:
asb_sample = df_both.copy()
asb_sample = asb_sample[['APN_ROW Segment','Street #', 'Street Name', 'County',
                         'ASB Assessment', 'ASB Sample Sent']]
asb_sample = asb_sample[(asb_sample['ASB Assessment'] > asb_sample['ASB Sample Sent'])]

asb_sample.set_index('APN_ROW Segment', inplace=True)
asb_sample.to_excel('ASB sample sent prior to ASB assessment.xlsx')

# ASB results prior to ASB sample sent date

In [ ]:
asb_res = df_both.copy()
asb_res = asb_res[['APN_ROW Segment','Street #', 'Street Name', 'County',
                   'ASB Sample Sent','ASB Results Received', 'ASB Results']]
asb_res = asb_res[(asb_res['ASB Sample Sent'].isna()) & (asb_res['ASB Results'].notna())]

asb_res.set_index('APN_ROW Segment', inplace=True)
asb_res.to_excel('ASB results prior to ASB sample sent date.xlsx')